<a href="https://colab.research.google.com/github/kartkerb02/Doc-Query-Agent/blob/main/NLQueryAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain
!pip install -q replicate
!pip install -q chromadb
!pip install -q pypdf
!pip install -q sentence-transformers

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA

In [3]:
import os
import sys

In [4]:
os.environ["REPLICATE_API_TOKEN"] = "r8_6bsJkXCY7GX2IBvTniWBEXsJIN1wyFC0uegtG"

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader

pdf_folder_path = f'./data/'

document=[]
for file in os.listdir(pdf_folder_path):
  file_path = pdf_folder_path+file
  if file.endswith(".pdf"):
    loader=PyPDFLoader(file_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    loader=Docx2txtLoader(file_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    loader=TextLoader(file_path)
    document.extend(loader.load())


In [6]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=80)
document_chunks=document_splitter.split_documents(document)
len(document_chunks)

241

In [7]:
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
db2 = Chroma.from_documents(document_chunks, embeddings, persist_directory="./chroma_db")
db2.persist()

my_retriever = db2.as_retriever(search_kwargs={'k':2})

In [10]:
# query = "What are the layers in a transformer block?"
# db2.similarity_search_with_score(query, k=3, fetch_k=1)

In [11]:
from langchain.llms import Replicate

llm = Replicate(
    model="meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    model_kwargs={"temperature": 0.6, "max_length": 600, "top_p": 1},
)

In [12]:
# memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [13]:
# custom_template = """Using the given conversation and supporting documents, answer the question to the best of your knowledge. Also privide citings and refernces to the information you use. If you do not know the answer reply with 'I am sorry'.
# Chat History: {chat_history}
# Question: {question}"""

# CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

In [14]:
# pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
#                                              retriever = db2.as_retriever(),
#                                              chain_type="stuff",
#                                              condense_question_prompt=CUSTOM_QUESTION_PROMPT,
#                                              memory=memory)

In [15]:
# print('---------------------------------------------------------------------------------')
# print('Welcome to the NLQueryAgent. You are now ready to start interacting with your documents')
# print('---------------------------------------------------------------------------------')

# while True:
#   query=input(f"Prompt: ")
#   if query == "exit" or query == "quit" or query == "q" or query == "f":
#     result = pdf_qa("question": "Please create a correct elaborate summary of our conversation history with key points")
#     print(f"Summary: " + result["answer"])
#     print('Exiting')
#     sys.exit()
#   if query == '':
#     continue
#   result = pdf_qa({"question": query})
#   print(f"Answer: " + result["answer"])



In [16]:
def qa(llm, chat_history, question):
  context = db2.similarity_search(question, k = 2)
  chunks = []
  for cont in context:
    chunks.append(["information: " + cont.page_content, "source: " + str(cont.metadata)])
  prompt = """
    QUESTION: {question}
    Context: {context}
    Chat History: {chat_history}


    Say Hi! The main goal is to answer the QUESTION using the chat history and context in less than 400 characters
    Give page and source name from the contect of the information used
    """
  final_question = prompt.format(question=question, context=context, chat_history=chat_history)
  # print(final_question)
  return llm(final_question)

In [19]:
print('---------------------------------------------------------------------------------')
print('Welcome to the NLQueryAgent. You are now ready to start interacting with your documents')
print('---------------------------------------------------------------------------------')
chat_history = []

while True:
  query=input(f"Prompt: ")
  if query == "exit" or query == "quit" or query == "q" or query == "f":
    q = "Please create a correct elaborate summary of our conversation history with key points"
    ans = qa(llm, chat_history, q)
    print(f"Summary: " + ans)
    print('Exiting...')
    break

  if query == '':
    continue

  ans = qa(llm, chat_history, query)
  chat_history.append(("question: " + query, "response: " + ans))
  print(f"Answer: " + ans)

---------------------------------------------------------------------------------
Welcome to the NLQueryAgent. You are now ready to start interacting with your documents
---------------------------------------------------------------------------------
Prompt: What is penn state bank
Answer:  Hello! The Penn Treebank is a classic dataset in natural language processing (NLP), originally annotated for syntactic parsing. It has been used in various research studies, including Emami and Jelinek (2004) and Mikolov and Zweig (2012). The dataset contains sentences with various grammatical structures, and is often used to evaluate the perplexity of language models. Perplexity refers to the difficulty of predicting a sentence or phrase, given its context. In NLP, perplexity is typically measured using a variety of metrics, such as language
Prompt: What are types of transformers
Answer:  Hi! There are two main types of transformers, namely:

1. Recurrent Neural Networks (RNNs): These include Long

ReplicateError: ignored